# Анализ текстовых сообщений в голосовании

In [1]:
# !pip install gensim
import pandas as pd
import numpy as np
import gensim

import warnings
warnings.simplefilter('ignore')

In [2]:
voting_comments = pd.read_excel('geo_comment.xlsx')
voting_comments.head()

,x,y,comment_class,multiplier,comment
0,37.612416,55.777454,-1,1,Во все разделы правил землепользования и застр...
1,37.612416,55.777454,-1,1,На основании вступившего в законную силу судеб...
2,37.603298,55.742108,-1,1,Внести в Проект правил землепользования и заст...
3,37.558526,55.728758,-1,1,Учитывая социальную значимость проекта строите...
4,37.566431,55.731794,-1,1,Учитывая социальную значимость проекта строите...


## Нормалиация текстов

In [6]:
import re
from functools import lru_cache

NUM = re.compile('^[\d\.,\:]{4,}$')  # 4 и больше цифры разделённые запятой, точкой или двоеточием

STOP_WORDS = set("""а,ах,без,более,больше,будет,будто,бы,был,была,были,было,
быть,в,вам,вас,вдруг,ведь,весь,во,вот,впрочем,все,всегда,всего,всех,всю,вы,г,
где,говорил,да,даже,два,для,до,другой,его,ее,её,ей,ему,если,есть,еще,ещё,ж,же,
жизнь,за,зачем,здесь,и,из,из-за,или,им,иногда,их,к,кажется,как,какая,какой,ко,
когда,конечно,которого,которые,кто,куда,ли,либо,лучше,между,меня,мне,много,может,
можно,мой,моя,мы,на,над,надо,наконец,нас,наш,не,него,нее,неё,ней,нельзя,нет,ни,
нибудь,никогда,ним,них,ничего,но,ну,о,об,один,однако,он,она,они,оно,опять,от,
ох,очень,перед,по,под,после,потом,потому,почти,при,про,раз,разве,с,сам,свое,свою,
себе,себя,сейчас,сказал,сказала,сказать,со,совсем,так,также,такой,там,те,
тебя,тем,теперь,то,тогда,того,тоже,той,только,том,тот,три,тут,ты,у,уж,уже,хорошо,
хоть,хотя,чего,чей,человек,чем,через,что,чтоб,чтобы,чуть,чье,чья,эта,эти,это,этого,
этой,этом,этот,эту,я,который,просто,точно,причём,никак,практически,вообще,лишь,именно,
наверное,как-то,что-тотам,вроде,таки,поскольку,кроме,что-то,ваш,свой,твой,мочь,самый,стать,
иметь,например,каждый,ве,лю,пр,др,ла,чо,що""".replace("\n", "").replace(" ", "").split(","))

@lru_cache(maxsize=1024)
def is_normal_word(word):
    if word: 
        word = word.strip(' ,-;!?*»«,)("	')
        if len(word) > 2:
            if word not in STOP_WORDS and not NUM.match(word):
                return True
    return False

In [7]:
# Удаляем дубликаты!
comments = voting_comments.comment.drop_duplicates().map(lambda comment: comment.replace('\xa0', ' '))
print("Было:", len(voting_comments))
print("Стало:", len(comments))

Было: 70382
Стало: 7335


In [8]:
###############################################################################
# Лемматизатор на основе Pymorphy2 с кешом и стоп-словами
# !pip install nltk
# nltk.download('punkt')
import nltk

In [9]:
import pymorphy2
from nltk.tokenize import word_tokenize

class TextNormalizer(object):
    def __init__(self):
        self.cache = {}
        self.morph = pymorphy2.MorphAnalyzer()
    
    def __call__(self, word):
        word = word.lower()
        if word not in self.cache:
            try:
                self.cache[word] = self.morph.parse(word)[0].normal_form
            except:
                self.cache[word] = None
                
        return self.cache[word]
        
    def lemmatize(self, text):
        return map(self, filter(is_normal_word, word_tokenize(text)))
    
    def get_normalized(self, text):
        words = word_tokenize(text)
        return " ".join(self.lemmatize(text))

In [10]:
# Нормализуем текст
normalizer = TextNormalizer()

In [11]:
# Пробуем лематизировать текст первого сообщения
text = voting_comments.comment[0]
print(text)
print('-' * 10)
print(normalizer.get_normalized(text))

Во все разделы правил землепользования и застройки г.Москвы (текстовые части и графические схемы) необходимо внести изменения по земельному участку с кадастровым номером 77:01:0004002:188 с адресом первый Самотёчный пер., вл. 17Б (в ПЗЗ территориальная зона №2034561) и исключить из этого земельного участка часть моей общей долевой собственности - земельного участка многоквартирного дома 17А по 1-му Самотёчному пер. площадью 650,5 кв.м с точками 1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-1 с координатами: 12316.39 и 6983.68; 123224 6988.91; 12316,45 и 7009,60; 12328.04 и 7013.46;12327.73 и 7014.41; 12330.06 и 7015.18; 12330.37 и 7014.24; 12335.17 и 7015.83; 12330.82 и 7028.61; 12304.74 и 7021.28; 12302.02 и 7012.47; 12303.22 и 7005.49; 12297.24 и 7004.18; 12297ю96 и 7000.60; 12311.23 и 7003.64; 12316.39 и 6983.68. - На основании вступившего в законную силу судебного решения по делу № А40-51937/2011 от 11.06.2015 о ничтожности заключённого 18.06.2007 договора аренды земельного участка с адре

In [12]:
###############################################################################
# Лематизатор на основе Mystem3, возвращает Counter {слово: число}
# !pip install pymystem3
from collections import Counter
from pymystem3 import Mystem
mystem = Mystem()

def text2bow_mystem3(text):
    return Counter(filter(is_normal_word, mystem.lemmatize(text)))

###############################################################################
# Лематизатор на основе rutermextract, возвращает сразу посчитанный мешок слов
# Умеет словосочетания и сам фильтрует стоп-слова. Очень медленный
# !pip install pymorphy2
# !pip install rutermextract
from rutermextract import TermExtractor
term_extractor = TermExtractor()

def text2bow_rutermextract(text):
    return Counter({
        term.normalized: term.count
        for term in term_extractor(text)
    })

###############################################################################
# Гибридный лемматизатор
# Просто суммирует результат двух лемматизаторов, возвращает генератор списка
from itertools import chain, repeat

def lemmatize(text):
    bow1 = text2bow_mystem3(text)
    bow2 = text2bow_rutermextract(text)
    bow1_words = set(bow1.keys())
    bow1.update({word: count for word, count in bow2.items() if word not in bow1_words})
    return chain.from_iterable(
        repeat(word, count) for word, count in bow1.items()
    )

In [13]:
# Пробуем лематизировать текст первого сообщения с попощью гибридного лемматизатора
text = voting_comments.comment[0]
print(text)
print('-' * 10)
print(' '.join(lemmatize(text)))

Во все разделы правил землепользования и застройки г.Москвы (текстовые части и графические схемы) необходимо внести изменения по земельному участку с кадастровым номером 77:01:0004002:188 с адресом первый Самотёчный пер., вл. 17Б (в ПЗЗ территориальная зона №2034561) и исключить из этого земельного участка часть моей общей долевой собственности - земельного участка многоквартирного дома 17А по 1-му Самотёчному пер. площадью 650,5 кв.м с точками 1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-1 с координатами: 12316.39 и 6983.68; 123224 6988.91; 12316,45 и 7009,60; 12328.04 и 7013.46;12327.73 и 7014.41; 12330.06 и 7015.18; 12330.37 и 7014.24; 12335.17 и 7015.83; 12330.82 и 7028.61; 12304.74 и 7021.28; 12302.02 и 7012.47; 12303.22 и 7005.49; 12297.24 и 7004.18; 12297ю96 и 7000.60; 12311.23 и 7003.64; 12316.39 и 6983.68. - На основании вступившего в законную силу судебного решения по делу № А40-51937/2011 от 11.06.2015 о ничтожности заключённого 18.06.2007 договора аренды земельного участка с адре

In [16]:
# Нормализуем весь корпус
from tqdm import tqdm_notebook

# _lemmatize = normalizer.lemmatize  # Быстрый лемматизатор на PyMorphy2
_lemmatize = lemmatize  # Медленный гибридный лемматизатор 

comments_doc = [
    list(_lemmatize(comment))
    for comment in tqdm_notebook(comments)
]

HBox(children=(IntProgress(value=0, max=7335), HTML(value='')))

In [472]:
# Сохранение/загрузка полного корпуса, чтобы избежать повторной лемматизации
# В репозитории full_corpus.pickle.zip
import pickle
# pickle.dump(comments_doc, open("full_corpus.pickle", "wb" ))
comments_doc = pickle.load(open("full_corpus.pickle", "rb"))

In [473]:
# Создаём словарь
dictionary = gensim.corpora.Dictionary(comments_doc)
len(dictionary.token2id)

50039

In [474]:
# Удаляем слова встречающиеся реже чем в 10 сообщениях и чаще чем в в 2/3
dictionary.filter_extremes(no_below=50, no_above=1/10)
len(dictionary.token2id)

2273

In [426]:
# Сохраняем словарь
dictionary.save('comments.dict')

In [427]:
# Словаризируем и сохраняем корпус
corpus = list(map(dictionary.doc2bow, comments_doc))

In [428]:
# Сохраняем корпус
gensim.corpora.MmCorpus.serialize('comments.mm', corpus)

In [429]:
# Теперь можно легко восстановить
dictionary = gensim.corpora.Dictionary.load('comments.dict')
corpus = gensim.corpora.MmCorpus('comments.mm')

In [430]:
# TfIdf — тестируем с ним и без
# model_tfidf = gensim.models.TfidfModel(corpus, dictionary=dictionary)
# corpus = model_tfidf[corpus]

In [431]:
#  gensim.models.LdaModel?

In [445]:
# Создаём и сохраняем LDA модель
num_topics = 7
lda_model = gensim.models.LdaModel(
    corpus, 
    id2word=dictionary, 
    num_topics=num_topics, 
    alpha='auto', 
    eta='auto',
    random_state=100,
    iterations=100,
    passes=3
)

In [446]:
lda_model.save('comments.lda')
# lda_model = gensim.models.LdaModel.load('comments.lda')

In [447]:
# Топики
from operator import itemgetter

def print_topics(model, count=8):
    for (keyword_no, keywords) in model.show_topics(formatted=False):
        print('[{}]: {}'.format(
            keyword_no,
            ', '.join(map(itemgetter(0), keywords[:count]))
        ))
        
print_topics(lda_model)   

[0]: культурный, наследие, культурное наследие, охранный, природный, карта, квартал, сохранение
[1]: мнение, участник, свобода, муниципальный, муниципальное округа, голосование, портал, электронный
[2]: административный, северо-западный, ознакомлять, южный, живописный, тушино, наземный, хороший
[3]: федеральный, федерация, капитальный строительство, орган, российский, пункт, книга, регламент
[4]: тыс, герой, панфиловец, обслуживание, герои панфиловцев, общественный, максимальный, процент
[5]: микрорайон, планировка, помещение, корпус, подземный, построить, этажный, паркинг
[6]: противоречить, доработка, отправлять, генеральный, фактический, спортивный, незаконный, бульвар


In [458]:
def get_topic(topic, count=7):
    return pd.DataFrame(
        lda_model.show_topic(topic, count),
        columns=['term', 'ratio']
    ).set_index('term').style.bar()

In [459]:
get_topic(0)

,ratio
term,
культурный,0.0180792
наследие,0.0166672
культурное наследие,0.0149464
охранный,0.00808084
природный,0.00732331
карта,0.00726317
квартал,0.00721512


In [460]:
get_topic(1)

,ratio
term,
мнение,0.0183778
участник,0.0156295
свобода,0.0148079
муниципальный,0.013748
муниципальное округа,0.0112308
голосование,0.0112238
портал,0.0104105


In [461]:
get_topic(2)

,ratio
term,
административный,0.0440447
северо-западный,0.0409594
ознакомлять,0.0313605
южный,0.0184311
живописный,0.0156856
тушино,0.0151951
наземный,0.0144546


In [462]:
get_topic(3)

,ratio
term,
федеральный,0.0118474
федерация,0.00866741
капитальный строительство,0.00745877
орган,0.00731293
российский,0.006973
пункт,0.00684872
книга,0.00590306


In [463]:
get_topic(4)

,ratio
term,
тыс,0.01471
герой,0.0130581
панфиловец,0.0122599
обслуживание,0.0112217
герои панфиловцев,0.0111315
общественный,0.0104335
максимальный,0.0102715


In [464]:
get_topic(5)

,ratio
term,
микрорайон,0.0167402
планировка,0.00989403
помещение,0.00981407
корпус,0.00970395
подземный,0.00923945
построить,0.00814348
этажный,0.00709729


In [465]:
get_topic(6)

,ratio
term,
противоречить,0.017044
доработка,0.0118122
отправлять,0.0102339
генеральный,0.00986893
фактический,0.00957297
спортивный,0.0088172
незаконный,0.00823774


In [454]:
# hdpmodel = gensim.models.HdpModel(corpus=corpus, id2word=dictionary)
# hdpmodel.show_topics(10)

In [455]:
# !pip install pyldavis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [456]:
pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)

/Users/ur001/.pyenv/versions/anaconda3-5.0.0/envs/netology1/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      37.112839        1       1 -0.092059  0.019836
0      16.165167        1       2 -0.116012  0.018579
6      12.057756        1       3 -0.115109  0.060997
5      11.774814        1       4  0.015671  0.080774
4       9.290406        1       5  0.030316  0.167264
1       8.626234        1       6 -0.079403 -0.294875
2       4.972784        1       7  0.356596 -0.052574, topic_info=     Category         Freq                            Term        Total  \
term                                                                      
271   Default  2627.000000                административный  2627.000000   
1824  Default  1275.000000                 северо-западный  1275.000000   
160   Default  1885.000000                      культурный  1885.000000   
190   Default   884.000000                     ознакомлять   884.000000   
161   Default  1729.000000                        наследие  1729.000000   
139   Default  1501.000000                   противоречить  1501.000000   
306   Default  1190.000000                      микрорайон  1190.000000   
175   Default  1558.000000             культурное наследие  1558.000000   
530   Default  1116.000000                          мнение  1116.000000   
1173  Default  1943.000000                        участник  1943.000000   
715   Default  1154.000000                   муниципальный  1154.000000   
680   Default   932.000000                         хороший   932.000000   
1594  Default   812.000000                         свобода   812.000000   
324   Default  1049.000000                             тыс  1049.000000   
1281  Default   655.000000                           южный   655.000000   
1319  Default   679.000000                           герой   679.000000   
61    Default   814.000000                        наземный   814.000000   
1600  Default   623.000000                      панфиловец   623.000000   
233   Default   828.000000  предельная плотность застройки   828.000000   
62    Default   953.000000                       подземный   953.000000   
1584  Default   522.000000                      живописный   522.000000   
841   Default  1176.000000                       доработка  1176.000000   
517   Default  1481.000000                     генеральный  1481.000000   
1585  Default   514.000000                          тушино   514.000000   
1603  Default   565.000000               герои панфиловцев   565.000000   
489   Default   860.000000                    обслуживание   860.000000   
2112  Default   570.000000            муниципальное округа   570.000000   
122   Default   826.000000                      отправлять   826.000000   
433   Default   555.000000                     голосование   555.000000   
408   Default   655.000000                        владение   655.000000   
...       ...          ...                             ...          ...   
1589   Topic7   389.325775                    южное тушино   424.108344   
264    Topic7   265.674455                        значимый   290.303462   
237    Topic7   178.499889              новодевичий проезд   195.051150   
226    Topic7   179.708685                          дашков   196.678570   
1492   Topic7   251.454500                      интересный   277.406669   
1824   Topic7  1112.738796                 северо-западный  1275.820309   
234    Topic7   295.890790          общая наземная площадь   329.384312   
238    Topic7   233.995709               медицинское центр   259.374822   
65     Topic7   369.067110                   зона развития   420.931332   
58     Topic7   225.476272                     погодинский   252.867755   
228    Topic7   186.659652                          фрунзе   206.822535   
1584   Topic7   426.129348                      живописный   522.696672   
57     Topic7   257.287825                     новодевичий   298.754450   
1585   Topic7   412.8

In [457]:
test = corpus[:500]
perplex = lda_model.bound(test)
per_word_perplex = np.exp2(-perplex / sum(cnt for document in test for _, cnt in document))
per_word_perplex

133.73086554109955

### perplexity при разных параметрах

- 249 (no_below=100, no_above=1/25, topics=15, auto/auto, passes=3, iterations=100)
- 318 (no_below=100, no_above=1/25, topics=15, None/None, passes=3, iterations=100)
- 27047 (no_below=10, no_above=1/25, topics=15, None/None, passes=3, iterations=100)
- 370 (no_below=10, no_above=1/10, topics=15, auto/auto, passes=3, iterations=100)
- 117 (no_below=50, no_above=1/10, topics=10, auto/auto, passes=3, iterations=100)
- 134 (no_below=50, no_above=1/10, topics=7, auto/auto, passes=3, iterations=100)